In [39]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from Models import LSTM, GRU
import time
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


Load and Prepare data to input to LSTM model

In [40]:
filepath = "RELIANCE_2010-01-012021-08-30.csv"
data = pd.read_csv(filepath, usecols=[0,4], names=['date', 'close'], header=0)
data = data.sort_values('date')
data['date'] = pd.to_datetime(data['date'])
data.head()

,date,close
0,2010-01-04,1075.50
1,2010-01-05,1070.70
2,2010-01-06,1088.00
3,2010-01-07,1106.05
4,2010-01-08,1103.15


In [41]:
len(data)

2895

In [42]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['date'], y=data['close'], mode='lines', name='closing price'))

fig.update_xaxes(range=["2009-11-01", "2021-11-01"])
fig.update_yaxes(range=[500, 2500])

In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['close'], mode='lines', name='closing price'))

fig.update_xaxes(range=["2009-11-01", "2021-11-01"])
fig.update_yaxes(range=[500, 2500])

In [44]:
def splitData(ts):

    # test_set_size = int(np.round(0.2*len(ts)));
    test_set_size = 365
    train_set = ts[:-test_set_size]    
    test_set = ts[-test_set_size:]

    return train_set, test_set


In [45]:
#Normalize data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

In [46]:
# Normalize the training, validation and test set

def normalize_data(train_set, test_set):
    train_norm = scaler.fit_transform(train_set.reshape(-1, 1))
    test_norm = scaler.transform(test_set.reshape(-1, 1))
    
    return train_norm, test_norm

In [47]:
window_size = 20
def prepareDataForTraining(seq):

    x_data = []
    y_data = []
    L = len(seq)
    for i in range(L-window_size):        
        window = seq[i:i+window_size]
        label = seq[i+window_size:i+window_size+1]
        x_data.append(window)
        y_data.append(label)
    return x_data, y_data    

In [48]:
train_set, test_set = splitData(data['close'].values)
train_norm, test_norm = normalize_data(train_set, test_set)

In [49]:
x_train, y_train = prepareDataForTraining(train_norm)
x_test, y_test = prepareDataForTraining(test_norm)

In [50]:
x_train = np.asarray(x_train).reshape(-1, window_size, 1)
y_train = np.asarray(y_train).reshape(-1, 1)
x_test = np.asarray(x_test).reshape(-1, window_size, 1)
y_test = np.asarray(y_test).reshape(-1, 1)

In [51]:
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

x_train.shape =  (2530, 20, 1)
y_train.shape =  (2530, 1)
x_test.shape =  (325, 20, 1)
y_test.shape =  (325, 1)


In [52]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [53]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

criterion = torch.nn.MSELoss(reduction='mean')

In [54]:
num_ensembles = 5
networks = []
optimisers = []

for i in range(num_ensembles):
    
    model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    networks.append(model)
    optimisers.append(optimiser)       


In [55]:
trained_networks = []

In [56]:
hist = np.zeros(num_epochs)
net = networks[0]
optim = optimisers[0]

for t in range(num_epochs):

    y_train_pred = net(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    hist[t] = loss.item()

    optim.zero_grad()
    loss.backward()
    optim.step()

print("Initial loss: ", hist[0])
print("Final loss", hist[num_epochs - 1])

trained_networks.append(net)

Initial loss:  0.3025701642036438
Final loss 0.00359428022056818


In [57]:
hist = np.zeros(num_epochs)
net = networks[1]
optim = optimisers[1]

for t in range(num_epochs):

    y_train_pred = net(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    hist[t] = loss.item()

    optim.zero_grad()
    loss.backward()
    optim.step()

print("Initial loss: ", hist[0])
print("Final loss", hist[num_epochs - 1])

trained_networks.append(net)

Initial loss:  0.19084058701992035
Final loss 0.0032631440553814173


In [58]:
len(trained_networks)

2

In [59]:
hist = np.zeros(num_epochs)
net = networks[2]
optim = optimisers[2]

for t in range(num_epochs):

    y_train_pred = net(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    hist[t] = loss.item()

    optim.zero_grad()
    loss.backward()
    optim.step()

print("Initial loss: ", hist[0])
print("Final loss", hist[num_epochs - 1])

trained_networks.append(net)

Initial loss:  0.3978939354419708
Final loss 0.003923181910067797


In [60]:
hist = np.zeros(num_epochs)
net = networks[3]
optim = optimisers[3]

for t in range(num_epochs):

    y_train_pred = net(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    hist[t] = loss.item()

    optim.zero_grad()
    loss.backward()
    optim.step()

print("Initial loss: ", hist[0])
print("Final loss", hist[num_epochs - 1])

trained_networks.append(net)

Initial loss:  0.3131072223186493
Final loss 0.0036833661142736673


In [61]:
hist = np.zeros(num_epochs)
net = networks[4]
optim = optimisers[4]

for t in range(num_epochs):

    y_train_pred = net(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    hist[t] = loss.item()

    optim.zero_grad()
    loss.backward()
    optim.step()

print("Initial loss: ", hist[0])
print("Final loss", hist[num_epochs - 1])

trained_networks.append(net)

Initial loss:  0.22378364205360413
Final loss 0.0037913902197033167


In [62]:
len(trained_networks)

5

#### Find uncertainty estimate on first test point

In [63]:
x_test[:1].shape

torch.Size([1, 20, 1])

In [64]:
outputs = []

for model in networks:
    output = model(x_test[:1])
    y_test_pred = scaler.inverse_transform(output.detach().numpy())
    outputs.append(y_test_pred)

ys = np.mean(outputs)
stds = np.std(outputs)*1.96

print("ys", ys)
print("std:", stds)

ys 1537.9269
std: 14.065630340576172


In [65]:
outputs

[array([[1544.184]], dtype=float32),
 array([[1524.5641]], dtype=float32),
 array([[1544.0029]], dtype=float32),
 array([[1539.5223]], dtype=float32),
 array([[1537.361]], dtype=float32)]

In [66]:
outputs = np.array(outputs).reshape(-1, 5)
outputs_df = pd.DataFrame(outputs)
outputs_df

,0,1,2,3,4
0,1544.18396,1524.564087,1544.00293,1539.522339,1537.360962


In [67]:
outputs_df['mean'] = outputs_df.apply(lambda x: x.mean(), axis=1)
outputs_df

,0,1,2,3,4,mean
0,1544.18396,1524.564087,1544.00293,1539.522339,1537.360962,1537.92688


In [68]:
np.mean(outputs_df.iloc[0, :]), np.std(outputs_df.iloc[0, 0:5])*1.96

(1537.9268595377605, 14.06563027272744)

#### Find uncertaintiy estimates of first 100 test points

In [69]:
y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())
test_gt = y_test[:, 0]

In [70]:
outputs = []

for model in networks:
    output = model(x_test[:10])
    y_test_pred = scaler.inverse_transform(output.detach().numpy())
    outputs.append(y_test_pred)

outputs = np.array(outputs).reshape(-1, 5)
outputs_df = pd.DataFrame(outputs)

outputs_df['mean'] = outputs_df.apply(lambda x: x.mean(), axis=1)
outputs_df['std'] = outputs_df.iloc[:, 0:5].std(axis=1)
outputs_df['95% interval'] = 1.96 * outputs_df['std']
outputs_df['80% interval'] = 1.28 * outputs_df['std']
outputs_df['Full interval'] = 3 * outputs_df['std']

In [71]:
outputs_df

,0,1,2,3,4,mean,std,95% interval,80% interval,Full interval
0,1544.183960,1507.807495,1452.268799,1424.094849,1414.266113,1468.524170,55.789162,109.346756,71.410126,167.367493
1,1402.618896,1406.618530,1420.280762,1428.421753,1427.189453,1417.025879,11.828277,23.183422,15.140194,35.484829
2,1524.564087,1485.718506,1440.046143,1432.563477,1435.391846,1463.656860,40.371964,79.129051,51.676113,121.115891
3,1422.134521,1423.556763,1434.754272,1437.497681,1430.531860,1429.695068,6.745688,13.221549,8.634480,20.237064
4,1544.002930,1520.989136,1476.656616,1445.121704,1424.845337,1482.323120,50.048000,98.094086,64.061440,150.143997
5,1405.223145,1402.397827,1412.611206,1422.660522,1426.845093,1413.947632,10.648471,20.871004,13.630042,31.945412
6,1539.522339,1508.402832,1461.069946,1434.989014,1422.550659,1473.307007,49.523754,97.066559,63.390404,148.571259
7,1406.717773,1406.061279,1417.178101,1425.356201,1425.893311,1416.241455,9.636528,18.887596,12.334756,28.909584
8,1537.360962,1515.947998,1474.105347,1446.517212,1428.030884,1480.392456,45.930386,90.023560,58.790894,137.791153
9,1407.973755,1404.328369,1412.498291,1419.673706,1421.518188,1413.198486,7.375855,14.456676,9.441094,22.127567


In [72]:
outputs_df['test GT'] = test_gt[:10]
outputs_df.head()

,0,1,2,3,4,mean,std,95% interval,80% interval,Full interval,test GT
0,1544.183960,1507.807495,1452.268799,1424.094849,1414.266113,1468.524170,55.789162,109.346756,71.410126,167.367493,1496.449951
1,1402.618896,1406.618530,1420.280762,1428.421753,1427.189453,1417.025879,11.828277,23.183422,15.140194,35.484829,1435.949951
2,1524.564087,1485.718506,1440.046143,1432.563477,1435.391846,1463.656860,40.371964,79.129051,51.676113,121.115891,1459.400024
3,1422.134521,1423.556763,1434.754272,1437.497681,1430.531860,1429.695068,6.745688,13.221549,8.634480,20.237064,1440.750000
4,1544.002930,1520.989136,1476.656616,1445.121704,1424.845337,1482.323120,50.048000,98.094086,64.061440,150.143997,1408.900024


In [73]:
# np.std(outputs_df.iloc[0, [0,1,2,3,4]], ddof=1)*1.96

In [74]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=outputs_df.index, y=outputs_df['mean'], mode='lines', name='preds mean'))
fig.add_trace(go.Scatter(x=outputs_df.index, y=outputs_df['test GT'], mode='lines', name='test ground truth'))
fig.add_trace(go.Scatter(x=outputs_df.index, y=(outputs_df['mean'] + outputs_df['Full interval']), mode='lines', name='upper limit'))
fig.add_trace(go.Scatter(x=outputs_df.index, y=(outputs_df['mean'] - outputs_df['Full interval']), mode='lines', name='lower limit'))

In [78]:
y_test[:20]

array([[1496.45  ],
       [1435.95  ],
       [1459.4   ],
       [1440.75  ],
       [1408.9   ],
       [1433.7001],
       [1441.25  ],
       [1431.55  ],
       [1424.05  ],
       [1445.5499],
       [1472.25  ],
       [1464.4   ],
       [1520.35  ],
       [1535.7   ],
       [1541.65  ],
       [1579.8   ],
       [1581.7001],
       [1569.5   ],
       [1537.1499],
       [1572.15  ]], dtype=float32)

In [83]:
y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
y_train[-15:]

array([[ 968.5   ],
       [ 917.7   ],
       [1017.95  ],
       [ 884.05  ],
       [ 943.4   ],
       [1082.25  ],
       [1066.2001],
       [1065.6   ],
       [1030.4501],
       [1113.75  ],
       [1080.45  ],
       [1077.4501],
       [1206.1   ],
       [1192.15  ],
       [1219.9501]], dtype=float32)

In [75]:
# for model, optimiser in models_ops:

#     hist = np.zeros(num_epochs)
#     for t in range(num_epochs):

#         y_train_pred = model(x_train)

#         loss = criterion(y_train_pred, y_train_lstm)
#         hist[t] = loss.item()

#         optimiser.zero_grad()
#         loss.backward()
#         optimiser.step()

#     print("Initial loss: ", hist[0])
#     print("Final loss", hist[num_epochs - 1])